In [ ]:
# 1) Set station and date window
from pathlib import Path

STATION = "senator_beck"
START_DATE = "2026-01-01"
END_DATE = "2026-01-31"

REPO_ROOT = Path.cwd()
if not (REPO_ROOT / "scripts").exists():
    REPO_ROOT = REPO_ROOT.parent

OUTDIR = REPO_ROOT / "results" / "issw_playground"
print("REPO_ROOT:", REPO_ROOT)
print("Station:", STATION)
print("Window:", START_DATE, "to", END_DATE)
print("Output folder:", OUTDIR)


In [ ]:
# 2) Run the scenario analysis script
import subprocess
import sys

cmd = [
    sys.executable,
    str(REPO_ROOT / "scripts" / "issw_variable_importance.py"),
    "--station", STATION,
    "--start", START_DATE,
    "--end", END_DATE,
]

print("Running:", " ".join(cmd))
subprocess.run(cmd, cwd=REPO_ROOT, check=True)
print("Done.")


In [ ]:
# 3) Load and inspect the main metrics table
import pandas as pd

metrics_path = OUTDIR / f"variable_importance_{STATION}_{START_DATE}_{END_DATE}.csv"
df_metrics = pd.read_csv(metrics_path)
print("Loaded:", metrics_path)
print("Rows:", len(df_metrics))
display(df_metrics.head(10))


In [ ]:
# 4) Load and inspect decision-impact + implementation status
decision_path = OUTDIR / f"decision_impact_hours_{STATION}_{START_DATE}_{END_DATE}.csv"
impl_path = OUTDIR / f"implementation_status_{STATION}_{START_DATE}_{END_DATE}.csv"

df_decision = pd.read_csv(decision_path)
df_impl = pd.read_csv(impl_path)

print("Loaded:", decision_path)
display(df_decision.head(12))

print("\nImplementation check:")
display(df_impl)


In [ ]:
# 5) Show top scenarios by impact (largest first)
cols = [
    "experiment",
    "family",
    "RMSE",
    "dRMSE_all",
    "dRMSE_cloud",
    "dRMSE_night",
    "dGRADRMSE_all",
    "ImpactScore",
]

top = df_metrics.sort_values("ImpactScore", ascending=False)[cols].head(12)
display(top)

print("Most impactful scenario:")
display(top.head(1))


In [ ]:
# 6) Display key plots
from IPython.display import Image, display

plot_names = [
    f"variable_importance_meteogram_{STATION}_{START_DATE}_{END_DATE}.png",
    f"variable_importance_plot_{STATION}_{START_DATE}_{END_DATE}.png",
    f"decision_impact_plot_{STATION}_{START_DATE}_{END_DATE}.png",
    f"mental_model_compare_{STATION}_{START_DATE}_{END_DATE}.png",
]

for name in plot_names:
    path = OUTDIR / name
    print("\n", name)
    if path.exists():
        display(Image(filename=str(path)))
    else:
        print("Missing:", path)
